<a href="https://colab.research.google.com/github/obscraft23/pybedrock/blob/main/demo/blockentity_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TL;DL**

This notebook is a demonstration of pybedrock.
Here you can create a flat mincraft world with a custom mob spawner from JSON files.

* author: obscraft23<br>
https://github.com/obscraft23

* pybedrock github page<br>
https://github.com/obscraft23/pybedrock


**Reference**

* How to make a custom spawner (Japanese)<br>
https://www.youtube.com/watch?v=ZbRscTGPVA0

Install pybedrock

In [ ]:
!pip install pybedrock

Download template json files.

In [ ]:
%cd /content
%rm -rf test
%mkdir -p test
%cd test

!wget -q "https://drive.google.com/uc?export=download&id=1fC1s6vOwoM8XzLEfEQl8q52Bm4NdiZ7c" -O custom.spawner.json
!wget -q "https://drive.google.com/uc?export=download&id=11hE1QqYihUAirrWejJkWRNn_EeXTgPZi" -O subchunk.nbt.json
!wget -q "https://drive.google.com/uc?export=download&id=1ADIHaBlnOUcC06Jjl0gp5eFPlmU2ohAw" -O level.dat.json

/content
/content/test


import libraries

In [ ]:
import pybedrock as pb
import random,string
import numpy as np
import copy
import json
import os
from google.colab import files

Define randomname generator

In [ ]:
def randomname(n):
   randlst = [random.choice(string.ascii_letters + string.digits) for i in range(n)]
   return ''.join(randlst)

Create a new world directory

In [ ]:
worldfname = randomname(12)
os.system("mkdir -p "+worldfname)
os.system("mkdir -p "+worldfname+"/db")
os.system("mkdir -p "+worldfname+"/resource_packs")
os.system("mkdir -p "+worldfname+"/behavior_packs")

0

Load json files

In [ ]:
with open("custom.spawner.json", mode="rt", encoding="utf-8") as f:
	cspawner = json.load(f)

with open("subchunk.nbt.json", mode="rt", encoding="utf-8") as f:
	schunk = json.load(f)

with open("level.dat.json", mode="rt", encoding="utf-8") as f:
	level = json.load(f)

Show block entity json file as a dict<br>
(Pig, cow, and sheep will spwan with equal weight.)

In [ ]:
cspawner

[{'type': 'Compound',
  'value': {'Delay': {'type': 'Short', 'value': 277},
   'DisplayEntityHeight': {'type': 'Float', 'value': 3.6},
   'DisplayEntityScale': {'type': 'Float', 'value': 1.0},
   'DisplayEntityWidth': {'type': 'Float', 'value': 1.6},
   'EntityIdentifier': {'type': 'String', 'value': 'minecraft:zombie'},
   'MaxNearbyEntities': {'type': 'Short', 'value': 6},
   'MaxSpawnDelay': {'type': 'Short', 'value': 800},
   'MinSpawnDelay': {'type': 'Short', 'value': 200},
   'RequiredPlayerRange': {'type': 'Short', 'value': 16},
   'SpawnCount': {'type': 'Short', 'value': 4},
   'SpawnRange': {'type': 'Short', 'value': 4},
   'id': {'type': 'String', 'value': 'MobSpawner'},
   'isMovable': {'type': 'Byte', 'value': 1},
   'x': {'type': 'Int', 'value': 2},
   'y': {'type': 'Int', 'value': -60},
   'z': {'type': 'Int', 'value': 0},
   'SpawnPotentials': {'type': 'List',
    'value': {'type_List': 'Compound',
     'value_List': [{'Properties': {'type': 'Compound', 'value': {}},
   

Make a subchunk data to which the spawner is put

In [ ]:
sc = np.zeros([16,16,16],dtype="int")
sc.fill(3)
sc[0] = 0
sc[1] = 1
sc[2] = 1
sc[3] = 2
sc[4][0][2] = 4

# 0->bedrock, 1->dirt, 2->grass, 3->air, 4-> mob spawner
# sc[4][0][2] means y=4,z=0,x=2 in the subchunk
# The target subchink is (0,0,-4), so the mob spawner is put to (2,60,0).

palette = pb.writeSubchunk(sc.tolist(),3,252)

Convert json files to binaries, and write them into world data.

In [ ]:
bin_schunk = pb.writeNBT(schunk)
bin_cspawner = pb.writeNBT(cspawner)
pb.writebinary(worldfname,'@0:0:0:47#-4',palette+bin_schunk)
pb.writebinary(worldfname,'@0:0:0:49',bin_cspawner)
pb.writebinary(worldfname,'@0:0:0:44',(0x28).to_bytes(1,byteorder="little"))

0

Show the current keys of the newly created world.

In [ ]:
pb.listkeys(worldfname)

[['@0:0:0:44', 1, 0, 0, 0, 44, -8, 1],
 ['@0:0:0:47#-4', 1951, 0, 0, 0, 47, -4, 1],
 ['@0:0:0:49', 538, 0, 0, 0, 49, -8, 1]]

Create levelname.txt

In [ ]:
with open(worldfname+"/levelname.txt","wt") as f:
  f.write("pybedroc demo")

Create level.dat

In [ ]:
levelheader = b'\n\x00\x00\x00\xe2\t\x00\x00'
bin_level = levelheader + pb.writeNBT(level)

with open(worldfname+"/level.dat","wb") as f:
  f.write(bin_level)

Make a mcworld file

In [ ]:
os.system("rm -rf "+worldfname+".zip")
os.chdir(worldfname)
os.system("zip -r ../"+worldfname+".zip "+"*")
os.chdir("../")
os.system("cp -r "+worldfname+".zip "+worldfname+".mcworld")

0

Download the world as a .mcworld

In [ ]:
files.download(worldfname+".mcworld")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>